In [ ]:
!pip install opencv-python

In [1]:
#Importing relevant libraries
import os
import cv2
import numpy as np
import pandas as pd
import csv

In [2]:
#Establishing filepaths and variables for later use
csv_file='automatically_annotated.csv'
base='/home/ubuntu/AffectNet/Automatically_Annotated'
output_dir='/home/ubuntu/AffectNet/AffectNet'

#Establishing empty lists and the desired size of our final images
input_size = (48, 48)
fname = []
face_x = []
face_y = []
width = []
height = []
expression = []



In [3]:
#Reading in the csv and verifying it worked
table = pd.read_csv(csv_file, sep=',')
table.head()
print(len(table))

539607


In [5]:
#Dropping columns from the dataframe that we don't need
table = table.drop(labels=["facial_landmarks", "valence", "arousal"], axis=1)

table.head()

,subDirectory_filePath,face_x,face_y,face_width,face_height,expression
0,324/27e086ee2eabdde2a009ce6d653862fd7d25097db9...,9,9,195,195,1
1,213/b7a2d66132c29a1ce5096a5e1be2a19f0e78f89e8c...,48,48,325,325,0
2,292/1e07c59017e7560116db9f359a03ec7a1bd1d71051...,106,106,709,709,1
3,1453/e7bf4ba69a82b8de5d6f37604fb35272b4d3e4eff...,15,15,210,210,2
4,1078/3b3277db30720c3621427a38567ba2ad6267c3a59...,23,23,157,157,1


In [6]:
#Sorting the table by expression for the next step
table = table.sort_values(by="expression")

table.head()

,subDirectory_filePath,face_x,face_y,face_width,face_height,expression
539606,1408/4f7253e6833a6f9d782e29587eddec4876aed7490...,37,37,253,253,0
321576,108/f40a5a727f960d7083b5a8ac706f6bec5af014dd82...,179,179,1904,1904,0
423083,1117/19feb7fb51e8768dffbc028e9540e235f828fd10c...,29,29,198,198,0
114451,567/1a1004df46b179ff42f4d645fb2dff706915b86906...,131,131,874,874,0
321580,228/bf08f6c11667cfe4e86a987b683b6945acb479f063...,38,38,254,254,0


In [7]:
#Dropping emotions will small sample sizes from the dataframe and verifying it worked
#We are left with: Neutral - 0, Happy - 1, Sad - 2, Surprise - 3, Anger - 6
to_drop = [4, 5, 7, 8, 9, 10]

for i in to_drop:
    table.drop(table[table["expression"] == i].index, inplace=True)

table.value_counts("expression")

expression
1    246235
0    143142
6     28000
2     20854
3     17462
dtype: int64

In [12]:
#Resetting the index of the dataframe so the next cell functions properly
table = table.reset_index(drop=True)

table.head()

,subDirectory_filePath,face_x,face_y,face_width,face_height,expression
0,1408/4f7253e6833a6f9d782e29587eddec4876aed7490...,37,37,253,253,0
1,108/f40a5a727f960d7083b5a8ac706f6bec5af014dd82...,179,179,1904,1904,0
2,1117/19feb7fb51e8768dffbc028e9540e235f828fd10c...,29,29,198,198,0
3,567/1a1004df46b179ff42f4d645fb2dff706915b86906...,131,131,874,874,0
4,228/bf08f6c11667cfe4e86a987b683b6945acb479f063...,38,38,254,254,0


In [13]:
#Creating an empty dataframe and some variables for counting our progress
final_document = pd.DataFrame(columns = ['face', 'expression'])
counter = 0
ten_thousands = 0

#Establishing a loop over the entire csv document
for i in range(0, len(table)):
        
    #Storing the relevant variables from the table
    fname = table['subDirectory_filePath'][i].split('/')[1]
    face_x = table['face_x'][i]
    face_y = table['face_y'][i]
    width = table['face_width'][i]
    height = table['face_height'][i]
    expression = table['expression'][i]
        
    #Reading in the image in grayscale
    image = cv2.imread(os.path.join(base, fname), 0)
        
    #Face extraction, if the image does not exist it moves on
    try:
        face = image[face_x : face_x + width, face_y : face_y + height]
    except:
        continue
        
    #Resizing the face and normalizing the pixel values
    resized_face = cv2.resize(face, input_size, interpolation = cv2.INTER_AREA)
    pixels = np.asarray(resized_face) 
    norm_face = pixels.astype('float32')
    norm_face /= 255.0
    norm_face = norm_face.reshape((48, 48, 1))
        
        
    #Adding the face and expression as a row to the dataframe
    final_document = final_document.append({'face': norm_face, 'expression': expression}, ignore_index=True)
    
    #Counting our progress through the dataframe
    counter += 1
    if counter > 9999:
        ten_thousands += 1
        print("{}0,000 faces processed.".format(ten_thousands))
        counter = 0

10,000 faces processed.
20,000 faces processed.
30,000 faces processed.
40,000 faces processed.
50,000 faces processed.
60,000 faces processed.
70,000 faces processed.
80,000 faces processed.
90,000 faces processed.
100,000 faces processed.
110,000 faces processed.
120,000 faces processed.
130,000 faces processed.
140,000 faces processed.
150,000 faces processed.
160,000 faces processed.
170,000 faces processed.
180,000 faces processed.
190,000 faces processed.
200,000 faces processed.
210,000 faces processed.
220,000 faces processed.
230,000 faces processed.
240,000 faces processed.
250,000 faces processed.
260,000 faces processed.
270,000 faces processed.


In [14]:
final_document.head()

,face,expression
0,"[[[0.5137255], [0.5058824], [0.50980395], [0.4...",0
1,"[[[0.16078432], [0.15294118], [0.14117648], [0...",0
2,"[[[0.52156866], [0.56078434], [0.53333336], [0...",0
3,"[[[0.02745098], [0.02745098], [0.02745098], [0...",0
4,"[[[0.7176471], [0.7176471], [0.7176471], [0.71...",0


In [15]:
#Grouping the final dataframe by expression (probably redundant)
final_document = final_document.groupby("expression")

#Creating five new dataframes (one for each expression)
df_neutral = final_document.get_group(0)
df_happy = final_document.get_group(1)
df_sad = final_document.get_group(2)
df_surprise = final_document.get_group(3)
df_anger = final_document.get_group(6)

In [16]:
#Surprise was the shortest of the five dataframes, so its length determines the max length
max_len = len(df_surprise)
train_len = int(max_len * 0.7)
test_len = int(max_len * 0.85)

#Sampling from the other four emotions so we are left with five dataframes of equal length
neutral_sample = df_neutral.sample(max_len)
happy_sample = df_happy.sample(max_len)
sad_sample = df_sad.sample(max_len)
anger_sample = df_anger.sample(max_len)

In [17]:
#Creating the training dataframe using the variables defined in the previous cell
train_df = pd.concat([neutral_sample[:train_len],
                     happy_sample[:train_len],
                     sad_sample[:train_len],
                     anger_sample[:train_len],
                     df_surprise[:train_len]])

#Shuffling the new dataframe
train_df = train_df.sample(frac=1).reset_index(drop=True)

#This process is repeated twice for the test and validation dataframes
test_df = pd.concat([neutral_sample[train_len:test_len],
                     happy_sample[train_len:test_len],
                     sad_sample[train_len:test_len],
                     anger_sample[train_len:test_len],
                     df_surprise[train_len:test_len]])

test_df = test_df.sample(frac=1).reset_index(drop=True)

valid_df = pd.concat([neutral_sample[test_len:],
                     happy_sample[test_len:],
                     sad_sample[test_len:],
                     anger_sample[test_len:],
                     df_surprise[test_len:]])

valid_df = valid_df.sample(frac=1).reset_index(drop=True)

In [18]:
#Verifying that our dataframes are the correct length
print(len(train_df))
print(len(test_df))
print(len(valid_df))

36780
7880
7885


In [44]:
#Small helper function that creates an empty numpy array of the correct shape and fills it with faces from the dataframe
def array_maker(dataframe):
    length = len(dataframe)
    array = np.empty((length, 48, 48, 1))
    for i in range(length):
        array[i] = np.asarray(dataframe['face'][i])
        
    return array    

In [49]:
#Running our function on the dataframes to create numpy arrays
train_x = array_maker(train_df)
test_x = array_maker(test_df)
valid_x = array_maker(valid_df)

In [58]:
#Creating numpy arrays from the expression column of the dataframes
#Note that at this point, none of the numpy arrays should be shuffled because
#They are no longer tethered to their other half
train_y = np.array(train_df['expression']).astype(int)
test_y = np.array(test_df['expression']).astype(int)
valid_y = np.array(valid_df['expression']).astype(int)

In [63]:
#Helper function that reassigns anger the value 4
#Interesting note: the model would error out if anger had the value 6 because
#That fell outside the expected range of 5 classes
def remapper(array):
    length = len(array)
    for i in range(length):
        if array[i] == 6:
            array[i] = 4
            
    return array        

In [66]:
#Running our function on all three numpy arrays, changing anger from 6 to 4
final_train_y = remapper(train_y)
final_test_y = remapper(test_y)
final_valid_y = remapper(valid_y)

In [65]:
#Double checking our y arrays are the correct length and properly balanced
unique, counts = np.unique(experiment_train_y, return_counts=True)
dict(zip(unique, counts))

{0: 7356, 1: 7356, 2: 7356, 3: 7356, 4: 7356}

In [26]:
#Defining the model
import tensorflow as tf
import tensorflow.keras as keras
from keras.models import Sequential, model_from_json
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout, BatchNormalization, Activation

classes = 5
num_features = 64

model = Sequential()

#Defining the layers
model.add(Conv2D(num_features, kernel_size=(3, 3), input_shape=(48, 48, 1)))
model.add(BatchNormalization())
model.add(Activation(activation='relu'))
model.add(Conv2D(num_features, kernel_size=(3, 3)))
model.add(BatchNormalization())
model.add(Activation(activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(classes, activation='softmax'))

In [67]:
#Compiling the model and running it for a few epochs
#Note the validation accuracy was already decent with no data augmentation
#Also note the time to train: roughly 14 minutes per epoch. A bit time-consuming to train in this environment
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history = model.fit(x=train_x, y=final_train_y, epochs=5, validation_data=(valid_x, final_valid_y), verbose=2)

Epoch 1/5
1150/1150 - 826s - loss: 1.0874 - accuracy: 0.5604 - val_loss: 1.0365 - val_accuracy: 0.6315
Epoch 2/5
1150/1150 - 815s - loss: 0.7225 - accuracy: 0.7218 - val_loss: 0.9155 - val_accuracy: 0.6548
Epoch 3/5
1150/1150 - 811s - loss: 0.6146 - accuracy: 0.7667 - val_loss: 0.6396 - val_accuracy: 0.7538
Epoch 4/5
1150/1150 - 813s - loss: 0.5501 - accuracy: 0.7884 - val_loss: 0.7603 - val_accuracy: 0.7106
Epoch 5/5
1150/1150 - 812s - loss: 0.5096 - accuracy: 0.8060 - val_loss: 0.5600 - val_accuracy: 0.7871
